In [ ]:
# pip install scikit-optimize

In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#            Machine Learning y Deep Learning
# ----------------------------------------------------------
#         Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#     Validación cruzada e Hypertuning con algoritmos 
#     GridSearchCV, RandomizedSearchCV, BayesSearchCV:
#          Clasificadores supervisados basados en 
#             Máquinas de Soporte Vectorial
# ==========================================================
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/enfermedades_cardiacas.csv'

# Cargar los datos en un DataFrame
df= pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(df.head())

In [ ]:
# Pre-procesamiento y preparación:

# Imputar los valores faltantes
df['talasemia'] = df['talasemia'].fillna(df['talasemia'].mean())
df['numerovasos'] = df['numerovasos'].fillna(df['numerovasos'].mean())

y = df['diagnostico'].map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})

# Etiquetas de las clases
etiquetas = ['sin enfermedad cardiaca', 'con enfermedad cardiaca']

# Seleccionar las columnas numéricas y convertir las categóricas a variables dummies
numerical_cols = ['edad','presion','colesterol','ritmocardiaco','numerovasos','caidast']
categorical_cols = ['sexo', 'talasemia', 'dolorpecho','electrocardiograma']

# Convertir las variables categóricas a dummies
df_categorical_dummies = pd.get_dummies(df[categorical_cols], drop_first=True)

# Crear la matriz X con las variables numéricas y categóricas recodificadas binarias
X = pd.concat([df[numerical_cols], df_categorical_dummies], axis=1)


In [ ]:
# Dividir el dataset en conjunto de entrenamiento y prueba
semilla = 
porcentaje_entrenamiento =  
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=1 - porcentaje_entrenamiento,
                                                    random_state=semilla)

In [ ]:
# Definir el modelo:
svc = SVC()

# Definir un grilla de hiperparámetros a probar 
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

In [ ]:
# Configurar la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(svc, param_grid, 
                           refit=True, 
                           verbose=3, 
                           cv=5)

# Entrenar el modelo
grid_search.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = grid_search.predict(X_test)

# Evaluar el modelo
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Exactitud en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred, 
                                                           target_names=etiquetas))

In [ ]:
# Búsqueda aleatoria con validación cruzada
semilla = 
random_search = RandomizedSearchCV(svc,
                                   param_distributions=param_grid, 
                                   n_iter=10, # muestras
                                   refit=True, # Reajuste con la muestra completa
                                   verbose=3, 
                                   cv=5, # k-folds
                                   random_state=semilla)

# Entrenar el modelo
random_search.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = random_search.predict(X_test)

# Evaluar el modelo
print("Mejores hiperparámetros:", random_search.best_params_)
print("Exactitud en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred, target_names=etiquetas))


In [ ]:
# Definir el espacio de búsqueda
search_spaces = {
    'C': (0.01, 1000, 'log-uniform'),
    'gamma': (0.001, 1, 'log-uniform'),
    'kernel': ['linear', 'rbf']
}
'''
    'C': [0.01, 0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
'''

# Configurar BayesSearchCV
opt = BayesSearchCV(
    estimator=SVC(),
    search_spaces=search_spaces,
    n_iter=32,  # Número de iteraciones de búsqueda
    cv=5,  # Validación cruzada
    n_jobs=-1,  # Uso de todos los núcleos disponibles
    verbose=3,  # Nivel de verbosidad
    random_state=42
)

# Entrenar el modelo
opt.fit(X_train, y_train)

# Evaluar el rendimiento
print("Mejores hiperparámetros:", opt.best_params_)
print("Exactitud en el conjunto de prueba:", opt.score(X_test, y_test))
